In [ ]:
endpoint = 
key = 

### OCR(Optical Character Recognition)/READ
* 문서에있는 텍스트 모조리 읽어버리기

In [ ]:
"""
This code sample shows Prebuilt Read operations with the Azure AI Document Intelligence client library.
The async versions of the samples require Python 3.8 or later.

To learn more, please visit the documentation - Quickstart: Document Intelligence (formerly Form Recognizer) SDKs
https://learn.microsoft.com/azure/ai-services/document-intelligence/quickstarts/get-started-sdks-rest-api?pivots=programming-language-python
"""

from azure.core.credentials import AzureKeyCredential
from azure.ai.documentintelligence import DocumentIntelligenceClient
from azure.ai.documentintelligence.models import AnalyzeDocumentRequest
import numpy as np

"""
Remember to remove the key from your code when you're done, and never post it publicly. For production, use
secure methods to store and access your credentials. For more information, see 
https://docs.microsoft.com/en-us/azure/cognitive-services/cognitive-services-security?tabs=command-line%2Ccsharp#environment-variables-and-application-configuration
"""
# endpoint = ""
# key = ""

def format_bounding_box(bounding_box):
    if not bounding_box:
        return "N/A"
    reshaped_bounding_box = np.array(bounding_box).reshape(-1, 2)
    return ", ".join(["[{}, {}]".format(x, y) for x, y in reshaped_bounding_box])

def analyze_read(file_path):
    document_intelligence_client  = DocumentIntelligenceClient(
            endpoint=endpoint, credential=AzureKeyCredential(key)
        )
    if "http" in file_path:
        # TODO: URL 파일처리
        # sample document (URL)
        formUrl = file_path
        poller = document_intelligence_client.begin_analyze_document(
            "prebuilt-read", AnalyzeDocumentRequest(url_source=formUrl)
        )
        result = poller.result()

    elif not "http" in file_path:
        # TODO: 로컬파일 경로가져오기
        # smaple document (local)
        local_path = file_path
        with open(local_path, "rb") as f:
            poller = document_intelligence_client.begin_analyze_document(
                "prebuilt-read",
                body= f
            )
        result = poller.result()

    # print(result.pages[0]["words"])
    # a = result.pages[0]["words"]

    # for i in a:
    #     print(i["polygon"])


    print ("Document contains content: ", result.content)

    # for idx, style in enumerate(result.styles):
    #     print(
    #         "Document contains {} content".format(
    #             "handwritten" if style.is_handwritten else "no handwritten"
    #         )
    #     )

    # for page in result.pages:
    #     print("----Analyzing Read from page #{}----".format(page.page_number))
    #     print(
    #         "Page has width: {} and height: {}, measured with unit: {}".format(
    #             page.width, page.height, page.unit
    #         )
    #     )

    #     for line_idx, line in enumerate(page.lines):
    #         print(
    #             "...Line # {} has text content '{}' within bounding box '{}'".format(
    #                 line_idx,
    #                 line.content,
    #                 format_bounding_box(line.polygon),
    #             )
    #         )

    #     for word in page.words:
    #         print(
    #             "...Word '{}' has a confidence of {}".format(
    #                 word.content, word.confidence
    #             )
    #         )

    print("----------------------------------------")


if __name__ == "__main__":
    analyze_read(file_path="https://raw.githubusercontent.com/Azure-Samples/cognitive-services-REST-api-samples/master/curl/form-recognizer/sample-layout.pdf")
    analyze_read(file_path="../Document Intelligence/Document Intelligence실습파일/generaldoc-drillreport.pdf")



* 그라디오에 구현해보기

In [ ]:
import gradio as gr

def analyze_read(file_path):
    document_intelligence_client  = DocumentIntelligenceClient(
            endpoint=endpoint, credential=AzureKeyCredential(key)
        )
    if "http" in file_path:
        # TODO: URL 파일처리
        # sample document (URL)
        formUrl = file_path
        poller = document_intelligence_client.begin_analyze_document(
            "prebuilt-read", AnalyzeDocumentRequest(url_source=formUrl)
        )
        result = poller.result()

        return result.content

    elif not "http" in file_path:
        # TODO: 로컬파일 경로가져오기
        # smaple document (local)
        local_path = file_path
        with open(local_path, "rb") as f:
            poller = document_intelligence_client.begin_analyze_document(
                "prebuilt-read",
                body= f
            )
        result = poller.result()

        return result.content

with gr.Blocks() as demo:
    input_text_box_1 = gr.Textbox()
    button_box_1 = gr.Button()
    input_image_box_1 = gr.Image(type='filepath')
    output_box_1 = gr.Textbox()
    # output_image_box_1 = gr.Image()

    button_box_1.click(fn=analyze_read, inputs=input_text_box_1, outputs=output_box_1)
    input_image_box_1.change(fn=analyze_read, inputs=input_image_box_1, outputs=output_box_1)

demo.launch()

* 여러가지 섞어서 조합해보기

In [ ]:
import requests, uuid, json
import os
import azure.cognitiveservices.speech as speechsdk

from azure.core.credentials import AzureKeyCredential
from azure.ai.documentintelligence import DocumentIntelligenceClient
from azure.ai.documentintelligence.models import AnalyzeDocumentRequest
import numpy as np

def analyze_read(file_path):
    document_intelligence_client  = DocumentIntelligenceClient(
            endpoint=endpoint, credential=AzureKeyCredential(key)
        )
    if "http" in file_path:
        # TODO: URL 파일처리
        # sample document (URL)
        formUrl = file_path
        poller = document_intelligence_client.begin_analyze_document(
            "prebuilt-read", AnalyzeDocumentRequest(url_source=formUrl)
        )
        result = poller.result()
        return result.content

    elif not "http" in file_path:
        # TODO: 로컬파일 경로가져오기
        # smaple document (local)
        local_path = file_path
        with open(local_path, "rb") as f:
            poller = document_intelligence_client.begin_analyze_document(
                "prebuilt-read",
                body= f
            )
        result = poller.result()
        return result.content


def trans_text(text):
    # Add your key and endpoint
    key = 
    endpoint = 

    # location, also known as region.
    # required if you're using a multi-service or regional (not global) resource. It can be found in the Azure portal on the Keys and Endpoint page.
    location = "eastus"

    path = '/translate'
    constructed_url = endpoint + path

    params = {
        'api-version': '3.0',
        'from': 'en',
        'to': ['ko']
    }

    headers = {
        'Ocp-Apim-Subscription-Key': key,
        # location required if you're using a multi-service or regional (not global) resource.
        'Ocp-Apim-Subscription-Region': location,
        'Content-type': 'application/json',
        'X-ClientTraceId': str(uuid.uuid4())
    }

    # You can pass more than one object in body.
    body = [{
        'text': text
    }]

    request = requests.post(constructed_url, params=params, headers=headers, json=body)
    response = request.json()

    return response[0]["translations"][0]["text"]

speech_config = speechsdk.SpeechConfig(
        subscription="", region="eastus"
        )
audio_config = speechsdk.audio.AudioOutputConfig(use_default_speaker=True)

# The neural multilingual voice can speak different languages based on the input text.
speech_config.speech_synthesis_voice_name='ko-KR-HyunsuMultilingualNeural'

speech_synthesizer = speechsdk.SpeechSynthesizer(speech_config=speech_config, audio_config=audio_config)

# Get text from the console and synthesize to the default speaker.
print("Enter some text that you want to speak >")
text = trans_text(analyze_read(file_path="../Document Intelligence/Document Intelligence실습파일/generaldoc-drillreport.pdf"))

speech_synthesis_result = speech_synthesizer.speak_text_async(text).get()

if speech_synthesis_result.reason == speechsdk.ResultReason.SynthesizingAudioCompleted:
    print("Speech synthesized for text [{}]".format(text))
elif speech_synthesis_result.reason == speechsdk.ResultReason.Canceled:
    cancellation_details = speech_synthesis_result.cancellation_details
    print("Speech synthesis canceled: {}".format(cancellation_details.reason))
    if cancellation_details.reason == speechsdk.CancellationReason.Error:
        if cancellation_details.error_details:
            print("Error details: {}".format(cancellation_details.error_details))
            print("Did you set the speech resource key and region values?")

Enter some text that you want to speak >
